In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from joblib import load, dump
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from fastai import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback, ReduceLROnPlateau
from torchvision import models as md
from torch import nn
from torch.nn import functional as F
from torch.utils import model_zoo
# %pip install pretrainedmodels
import pretrainedmodels as pm
import re
import math
import collections
from functools import partial
from collections import Counter
import json
from os.path import join, isfile
# %pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=1)

In [ ]:
df = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')

# k-fold
df['fold'] = 0
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=8) # if not fix random_state, then save file
for idx, (train_idx, valid_idx) in enumerate(skf.split(df, df['diagnosis'])):
    df['fold'].iloc[valid_idx] = idx
    
# change id_code to absolute file path
PATH = '/kaggle/input/aptos2019-blindness-detection/'

def aptos_isfile(id_code):
    if isfile(PATH+'train_images/'+id_code+'.png'): return PATH+'train_images/'+id_code+'.png'
    if isfile(PATH+'train_images/'+id_code+'.jpeg'): return PATH+'train_images/'+id_code+'.jpeg'
    if isfile(PATH+'test_images/'+id_code+'.png'): return PATH+'test_images/'+id_code+'.png'
    if isfile(PATH+'test_images/'+id_code+'.jpeg'): return PATH+'test_images/'+id_code+'.jpeg'
    
df['id_code'] = df['id_code'].apply(lambda x: aptos_isfile(x))

# save df
df.to_csv('k-fold_df.csv', index=False)
df = pd.read_csv('/kaggle/working/k-fold_df.csv')

In [ ]:
df

In [ ]:
valid_fold = 1
train_idx = np.array(df[df['fold']!=valid_fold].index)
valid_idx = np.array(df[df['fold']==valid_fold].index)

In [ ]:
sz = 512
bs = 8
tfms = aug_transforms(do_flip=True, flip_vert=True)

data = ImageDataLoaders.from_df(
    df=df,
    path='/',
    valid_idx=valid_idx,
    fn_col='id_code',
    label_col='diagnosis',
    item_tfms=Resize(sz),
    batch_tfms=[*tfms, Normalize.from_stats(*imagenet_stats)],
    bs=bs,
    num_workers=8
)

In [ ]:
def qwk(y_pred, y):
    return torch.Tensor(cohen_kappa_score(torch.round(y_pred), y, weights='quadratic', device='cude'))

In [ ]:
learn = Learner(data, model, wd=0.02, metrics=[qwk], model_dir='/kaggle/working/models').to_fp16()

In [ ]:
callbacks = [
    SaveModelCallback(monitor='valid_loss', fname='best_model'),
    EarlyStoppingCallback(monitor='valid_loss', patience=5),
    ReduceLROnPlateau(monitor='valid_loss', patience=1, factor=0.2)
]

In [ ]:
learn.fit(n_epoch=20, lr=1e-4, cbs=callbacks)